In [20]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam


In [21]:
data = pd.read_csv('BS_data.csv')
data

,Company,Time,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Price
0,RHI,2013-12-01,1490.3,570.6,919.6,135.3,6.80,718.3,5.31,1.4,-274.3,41.99
1,RCL,2011-12-01,19804.4,11396.6,8407.8,217.5,38.74,7442.4,34.29,8507.2,8245.0,24.77
2,NVDA,2020-01-01,17315.0,5111.0,12204.0,612.0,19.93,11537.0,18.84,2643.0,-8254.0,236.43
3,STC,2012-12-01,1291.2,710.8,580.4,21.1,29.31,263.5,13.58,71.2,-137.4,26.00
4,XRAY,2010-12-01,3258.0,1348.0,1909.9,142.1,12.98,457.6,3.23,786.6,246.6,34.17
5,GWW,2016-12-01,5694.3,3788.5,1905.8,58.8,30.57,684.7,11.64,2247.1,1972.9,232.25
6,SKM,2015-12-01,24288.4,11223.5,13064.9,70.6,183.55,9386.2,132.93,7039.9,5790.9,20.15
7,BAX,2011-12-01,19073.0,12245.0,6828.0,560.3,11.74,3442.0,6.14,5195.0,2290.0,26.88
8,MAR,2012-12-01,6342.0,7627.0,-1285.0,312.3,-4.13,-3274.0,-10.53,2935.0,2847.0,37.27
9,SJM,2019-04-01,16711.3,8740.8,7970.5,113.7,70.08,-5059.2,-44.48,5959.9,5858.6,122.63


In [22]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)

# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:9] # Parameters
y = NpMatrix[:,9] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

Total Assets
Done
Total Liabilities
Done
Total Equity
Done
Total Shares Out. on Filing Date
Done
Book Value / Share
Done
Tangible Book Value
Done
Tangible Book Value Per Share
Done
Total Debt
Done
Net Debt
Done
X: [[1.63843386e-03 6.82611560e-04 2.81693214e-02 ... 1.34060792e-03
  2.92521072e-06 1.85568632e-01]
 [2.19349988e-02 1.36816489e-02 4.51864494e-02 ... 1.49174313e-03
  1.77752519e-02 2.00645972e-01]
 [1.91761262e-02 6.13437854e-03 5.38134120e-02 ... 1.41116898e-03
  5.52237995e-03 1.71446269e-01]
 ...
 [6.79067888e-03 4.97604015e-03 3.06086506e-02 ... 1.34050362e-03
  4.01380699e-03 1.89340224e-01]
 [2.35835177e-03 1.36510305e-03 2.83540777e-02 ... 1.34947368e-03
  1.52340795e-03 1.86742708e-01]
 [7.21358632e-04 8.08087212e-04 2.60515545e-02 ... 1.31145520e-03
  1.21500717e-03 1.86722887e-01]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 9
Total Number of Training instances: 5000


[ 41.99  24.77 236.43 ...  49.01   8.07  34.69]


Type y: <class 'nump

In [23]:
number_of_features = len(X[1])

#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=81, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.2))

    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [24]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 1000, 
                                 batch_size= 5)

In [25]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/1000
4000/4000 [==============================] - 1s 347us/sample - loss: 57.0672 - mae: 57.0671 - mse: 19957.1719
Epoch 2/1000
4000/4000 [==============================] - 1s 228us/sample - loss: 46.4479 - mae: 46.4479 - mse: 17792.1934
Epoch 3/1000
4000/4000 [==============================] - 1s 303us/sample - loss: 46.2991 - mae: 46.2990 - mse: 17734.0098
Epoch 4/1000
4000/4000 [==============================] - 1s 235us/sample - loss: 46.1344 - mae: 46.1344 - mse: 17656.3594
Epoch 5/1000
4000/4000 [==============================] - 1s 306us/sample - loss: 46.1604 - mae: 46.1604 - mse: 17668.1230
Epoch 6/1000
4000/4000 [==============================] - 1s 337us/sample - loss: 46.0499 - mae: 46.0499 - mse: 17696.8105
Epoch 7/1000
4000/4000 [==============================] - 1s 324us/sample - loss: 46.1816 - mae: 46.1816 - mse: 17710.8730
Epoch 8/1000
4000/4000 [==============================] - 1s 311us/sample - loss: 46.0750 - mae: 46.0750 - mse: 17699

Epoch 67/1000
4000/4000 [==============================] - 2s 486us/sample - loss: 45.1180 - mae: 45.1179 - mse: 17477.7832
Epoch 68/1000
4000/4000 [==============================] - 2s 377us/sample - loss: 45.1441 - mae: 45.1441 - mse: 17487.3184
Epoch 69/1000
4000/4000 [==============================] - 2s 410us/sample - loss: 45.0216 - mae: 45.0216 - mse: 17436.0957
Epoch 70/1000
4000/4000 [==============================] - 2s 458us/sample - loss: 45.0349 - mae: 45.0349 - mse: 17432.7754
Epoch 71/1000
4000/4000 [==============================] - 2s 400us/sample - loss: 45.1059 - mae: 45.1059 - mse: 17483.6035
Epoch 72/1000
4000/4000 [==============================] - 2s 451us/sample - loss: 45.0113 - mae: 45.0113 - mse: 17412.7266
Epoch 73/1000
4000/4000 [==============================] - 2s 501us/sample - loss: 45.0811 - mae: 45.0811 - mse: 17438.6875
Epoch 74/1000
4000/4000 [==============================] - 2s 509us/sample - loss: 45.0816 - mae: 45.0817 - mse: 17454.5547
Epoch 75

4000/4000 [==============================] - 2s 422us/sample - loss: 44.3452 - mae: 44.3452 - mse: 17289.3457
Epoch 133/1000
4000/4000 [==============================] - 2s 414us/sample - loss: 44.3564 - mae: 44.3564 - mse: 17265.7500
Epoch 134/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 44.4549 - mae: 44.4549 - mse: 17261.0293
Epoch 135/1000
4000/4000 [==============================] - 2s 397us/sample - loss: 44.3243 - mae: 44.3243 - mse: 17266.6699
Epoch 136/1000
4000/4000 [==============================] - 2s 402us/sample - loss: 44.0374 - mae: 44.0375 - mse: 17224.5371
Epoch 137/1000
4000/4000 [==============================] - 2s 379us/sample - loss: 44.2443 - mae: 44.2442 - mse: 17244.6191
Epoch 138/1000
4000/4000 [==============================] - 2s 413us/sample - loss: 44.3424 - mae: 44.3424 - mse: 17270.6641
Epoch 139/1000
4000/4000 [==============================] - 1s 366us/sample - loss: 44.3028 - mae: 44.3028 - mse: 17273.7812
Epoch 140/1000


4000/4000 [==============================] - 2s 402us/sample - loss: 43.8168 - mae: 43.8167 - mse: 17101.0195
Epoch 198/1000
4000/4000 [==============================] - 2s 410us/sample - loss: 43.7762 - mae: 43.7762 - mse: 17119.1328
Epoch 199/1000
4000/4000 [==============================] - 1s 375us/sample - loss: 43.7942 - mae: 43.7942 - mse: 17075.9727
Epoch 200/1000
4000/4000 [==============================] - 2s 381us/sample - loss: 43.8220 - mae: 43.8221 - mse: 17095.7969
Epoch 201/1000
4000/4000 [==============================] - 2s 385us/sample - loss: 43.7631 - mae: 43.7631 - mse: 17094.2715
Epoch 202/1000
4000/4000 [==============================] - 2s 406us/sample - loss: 43.7261 - mae: 43.7261 - mse: 17043.0176
Epoch 203/1000
4000/4000 [==============================] - 2s 409us/sample - loss: 43.6173 - mae: 43.6173 - mse: 17066.3438
Epoch 204/1000
4000/4000 [==============================] - 2s 400us/sample - loss: 43.9868 - mae: 43.9868 - mse: 17078.6074
Epoch 205/1000


4000/4000 [==============================] - 1s 362us/sample - loss: 43.2605 - mae: 43.2605 - mse: 16921.2188
Epoch 263/1000
4000/4000 [==============================] - 2s 456us/sample - loss: 43.2380 - mae: 43.2380 - mse: 16951.1387
Epoch 264/1000
4000/4000 [==============================] - 2s 388us/sample - loss: 43.1600 - mae: 43.1600 - mse: 16894.5762
Epoch 265/1000
4000/4000 [==============================] - 1s 374us/sample - loss: 43.2878 - mae: 43.2878 - mse: 16955.8223
Epoch 266/1000
4000/4000 [==============================] - 2s 387us/sample - loss: 43.2945 - mae: 43.2945 - mse: 16875.5918
Epoch 267/1000
4000/4000 [==============================] - 2s 384us/sample - loss: 43.3477 - mae: 43.3477 - mse: 16919.0234
Epoch 268/1000
4000/4000 [==============================] - 2s 410us/sample - loss: 43.3277 - mae: 43.3278 - mse: 16897.5566
Epoch 269/1000
4000/4000 [==============================] - 2s 387us/sample - loss: 43.1113 - mae: 43.1112 - mse: 16799.3906
Epoch 270/1000


...

4000/4000 [==============================] - 2s 408us/sample - loss: 41.2633 - mae: 41.2634 - mse: 15113.8037
Epoch 718/1000
4000/4000 [==============================] - 2s 380us/sample - loss: 40.8344 - mae: 40.8344 - mse: 15075.6270
Epoch 719/1000
4000/4000 [==============================] - 2s 404us/sample - loss: 41.1360 - mae: 41.1360 - mse: 15075.6152
Epoch 720/1000
4000/4000 [==============================] - 1s 355us/sample - loss: 41.3540 - mae: 41.3539 - mse: 15233.2324
Epoch 721/1000
4000/4000 [==============================] - 2s 386us/sample - loss: 41.2747 - mae: 41.2747 - mse: 15265.3906
Epoch 722/1000
4000/4000 [==============================] - 1s 356us/sample - loss: 41.1985 - mae: 41.1985 - mse: 15117.9307
Epoch 723/1000
4000/4000 [==============================] - 2s 408us/sample - loss: 41.3639 - mae: 41.3639 - mse: 15237.9814
Epoch 724/1000
4000/4000 [==============================] - 1s 361us/sample - loss: 41.2459 - mae: 41.2459 - mse: 15242.2480
Epoch 725/1000


4000/4000 [==============================] - 2s 389us/sample - loss: 41.0199 - mae: 41.0200 - mse: 15001.0479
Epoch 783/1000
4000/4000 [==============================] - 1s 361us/sample - loss: 41.1597 - mae: 41.1597 - mse: 15061.1973
Epoch 784/1000
4000/4000 [==============================] - 2s 401us/sample - loss: 41.2376 - mae: 41.2376 - mse: 15217.4414
Epoch 785/1000
4000/4000 [==============================] - 1s 373us/sample - loss: 41.0449 - mae: 41.0448 - mse: 15094.5381
Epoch 786/1000
4000/4000 [==============================] - 1s 366us/sample - loss: 41.2099 - mae: 41.2099 - mse: 15091.4521
Epoch 787/1000
4000/4000 [==============================] - 2s 410us/sample - loss: 41.0676 - mae: 41.0676 - mse: 15072.8174
Epoch 788/1000
4000/4000 [==============================] - 1s 364us/sample - loss: 40.9758 - mae: 40.9759 - mse: 14989.4414
Epoch 789/1000
4000/4000 [==============================] - 2s 387us/sample - loss: 41.0927 - mae: 41.0927 - mse: 15177.4434
Epoch 790/1000


4000/4000 [==============================] - 2s 407us/sample - loss: 40.8555 - mae: 40.8555 - mse: 14947.7305- loss: 37.9253 - 
Epoch 848/1000
4000/4000 [==============================] - 2s 440us/sample - loss: 40.7286 - mae: 40.7286 - mse: 14948.3428
Epoch 849/1000
4000/4000 [==============================] - 2s 380us/sample - loss: 40.7744 - mae: 40.7744 - mse: 14992.6260
Epoch 850/1000
4000/4000 [==============================] - 2s 421us/sample - loss: 41.0856 - mae: 41.0856 - mse: 15060.0088
Epoch 851/1000
4000/4000 [==============================] - 2s 475us/sample - loss: 40.7656 - mae: 40.7656 - mse: 15003.1865
Epoch 852/1000
4000/4000 [==============================] - 2s 447us/sample - loss: 40.9996 - mae: 40.9996 - mse: 15004.1650
Epoch 853/1000
4000/4000 [==============================] - 2s 426us/sample - loss: 41.0566 - mae: 41.0566 - mse: 14975.6709
Epoch 854/1000
4000/4000 [==============================] - 2s 513us/sample - loss: 40.8241 - mae: 40.8241 - mse: 14994.84

4000/4000 [==============================] - 2s 425us/sample - loss: 40.9229 - mae: 40.9229 - mse: 14930.4922
Epoch 913/1000
4000/4000 [==============================] - 1s 369us/sample - loss: 40.7201 - mae: 40.7201 - mse: 14889.0186
Epoch 914/1000
4000/4000 [==============================] - 1s 301us/sample - loss: 40.8898 - mae: 40.8898 - mse: 14955.6064
Epoch 915/1000
4000/4000 [==============================] - 2s 385us/sample - loss: 40.7651 - mae: 40.7651 - mse: 14894.0723
Epoch 916/1000
4000/4000 [==============================] - 2s 430us/sample - loss: 40.8802 - mae: 40.8802 - mse: 14932.7148
Epoch 917/1000
4000/4000 [==============================] - 2s 396us/sample - loss: 40.7153 - mae: 40.7153 - mse: 14901.0684
Epoch 918/1000
4000/4000 [==============================] - 1s 357us/sample - loss: 40.9408 - mae: 40.9408 - mse: 14911.2178
Epoch 919/1000
4000/4000 [==============================] - 1s 348us/sample - loss: 40.8735 - mae: 40.8735 - mse: 14915.3037
Epoch 920/1000


4000/4000 [==============================] - 1s 368us/sample - loss: 40.8717 - mae: 40.8718 - mse: 14849.7842
Epoch 978/1000
4000/4000 [==============================] - 1s 311us/sample - loss: 40.7509 - mae: 40.7509 - mse: 14866.1924
Epoch 979/1000
4000/4000 [==============================] - 1s 373us/sample - loss: 40.8036 - mae: 40.8036 - mse: 14933.5195
Epoch 980/1000
4000/4000 [==============================] - 1s 313us/sample - loss: 40.6472 - mae: 40.6472 - mse: 14894.2373
Epoch 981/1000
4000/4000 [==============================] - 2s 408us/sample - loss: 40.7160 - mae: 40.7160 - mse: 14969.4688
Epoch 982/1000
4000/4000 [==============================] - 1s 374us/sample - loss: 40.7865 - mae: 40.7865 - mse: 14927.8037
Epoch 983/1000
4000/4000 [==============================] - 1s 301us/sample - loss: 40.8234 - mae: 40.8234 - mse: 14978.0947
Epoch 984/1000
4000/4000 [==============================] - 1s 292us/sample - loss: 40.8279 - mae: 40.8279 - mse: 14817.1592
Epoch 985/1000


In [26]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-44.36649898 -38.38892684 -41.04799083 -37.11146395 -38.90490908]
Mean MAE: -39.96395793962479
Standard Deviation of the mean: 2.541516253487928
